In [23]:
import scanpy as sc
import pandas as pd
import numpy as np
import sklearn.metrics
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from collections import Counter

In [24]:
label = sc.read_csv('darmanis_label.csv',dtype="str")
adata = sc.read_csv("darmanis_counts.csv")
data = adata.to_df()

In [25]:
adata
# data

AnnData object with n_obs × n_vars = 466 × 22088

In [26]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

def build_dataset(adata_this,label_this,drop_number=1):



    adata=adata_this
    # sc.pp.normalize_total(adata, target_sum=1e4)
    # sc.pp.log1p(adata)
    label=label_this.to_df()

    print("Dataset Size {}".format(len(adata)))

    assert len(adata)==len(label),"Data Length {} Does not Match Label Length {}".format(len(adata),len(label))

    set_label=np.unique(label.values)
    print(set_label)

    #Find the rare cell class
    count=[]
    for label_index in range(len(set_label)):
        # print(label.values==set_label[label_index])
        sum_number=sum(label.values==set_label[label_index])
        count.append(sum_number[0])
        print("Label {}: {}".format(set_label[label_index],sum_number))
    print(count)
    print(np.array(count))
    print(np.argmin(count))
    min_index=np.argsort(count)[:drop_number]
    print(min_index)
    min_label=[]
    for i in min_index:
        min_label.append(set_label[i])
    print(min_label)
    print("Rare Cell Name {}: {}".format(min_label,[count[count_i] for count_i in min_index]))

    # Build Class Mapping for cell class
    label_map={}
    count_map=0
    for label_index in range(len(set_label)):
        if not label_index in min_index:
            label_map[set_label[label_index]]=count_map
            count_map+=1
    print(label_map)
    count_map_temp=count_map
    for label_index in range(len(set_label)):
        if label_index in min_index:
            label_map[set_label[label_index]]=count_map_temp
            count_map_temp+=1
    # label_map[min_label]=count_map
    print(label_map)

    test_extract_ood = label.isin(min_label)
    label_ood = label.loc[test_extract_ood.values]
    X_test_ood=adata.loc[test_extract_ood.values]
    # print(label_ood.values[0])
    adata_id = adata.loc[~test_extract_ood.values]
    label_id = label.loc[~test_extract_ood.values]

    X_train, X_test, y_train, y_test = train_test_split(adata_id,label_id,random_state=42,stratify=label_id)
    print(X_train)
    print(y_train)
    return X_train,X_test,y_train,y_test,X_test_ood,label_ood
X_train,X_test,y_train,y_test,X_test_ood,label_ood=build_dataset(data,label,drop_number=1)
true_label=np.zeros(len(X_test)+len(X_test_ood))
true_label[:len(X_test)]+=1

Dataset Size 466
['"OPC"' '"astrocytes"' '"endothelial"' '"fetal_quiescent"'
 '"fetal_replicating"' '"hybrid"' '"microglia"' '"neurons"'
 '"oligodendrocytes"']
Label "OPC": [18]
Label "astrocytes": [62]
Label "endothelial": [20]
Label "fetal_quiescent": [110]
Label "fetal_replicating": [25]
Label "hybrid": [46]
Label "microglia": [16]
Label "neurons": [131]
Label "oligodendrocytes": [38]
[np.int64(18), np.int64(62), np.int64(20), np.int64(110), np.int64(25), np.int64(46), np.int64(16), np.int64(131), np.int64(38)]
[ 18  62  20 110  25  46  16 131  38]
6
[6]
['"microglia"']
Rare Cell Name ['"microglia"']: [np.int64(16)]
{'"OPC"': 0, '"astrocytes"': 1, '"endothelial"': 2, '"fetal_quiescent"': 3, '"fetal_replicating"': 4, '"hybrid"': 5, '"neurons"': 6, '"oligodendrocytes"': 7}
{'"OPC"': 0, '"astrocytes"': 1, '"endothelial"': 2, '"fetal_quiescent"': 3, '"fetal_replicating"': 4, '"hybrid"': 5, '"neurons"': 6, '"oligodendrocytes"': 7, '"microglia"': 8}
                               1/2-SBSR

In [27]:
# label.columns = ['Label']
y_train.Label.value_counts()
status_dict = y_train['Label'].unique().tolist()
y_train['transfromed']=y_train['Label'].apply(lambda x : status_dict.index(x))
label_arr = y_train['transfromed'].values
print(label_arr)

[0 0 1 2 0 0 3 0 1 4 3 0 5 0 0 0 0 4 1 6 1 7 0 5 7 7 3 1 7 5 5 5 0 0 1 1 1
 5 4 3 5 5 2 5 0 0 3 1 4 0 3 4 0 0 1 3 5 3 3 6 5 0 3 7 4 0 5 5 7 5 2 7 0 5
 6 5 3 5 4 0 5 2 5 2 4 2 3 0 3 3 5 0 0 5 5 6 7 3 5 2 5 5 5 5 4 0 0 5 6 3 5
 3 3 6 3 7 0 3 5 3 0 7 0 5 0 5 0 1 0 1 0 5 7 0 7 3 5 0 0 1 5 0 2 1 5 4 1 7
 0 5 0 0 5 2 5 0 3 2 4 5 5 1 0 7 4 3 6 7 3 0 4 5 1 4 6 0 2 5 5 0 5 5 0 1 0
 4 7 3 7 0 5 3 7 5 3 0 5 3 1 5 1 0 1 0 0 3 0 7 3 7 6 1 5 1 0 5 5 3 7 0 1 5
 0 0 2 5 3 3 5 5 5 5 6 7 0 5 3 5 7 0 5 5 5 5 7 3 5 0 7 5 1 5 7 5 0 4 5 5 2
 0 7 0 0 5 0 5 0 5 3 5 5 0 6 3 3 4 2 5 0 7 3 4 7 0 4 5 1 5 3 6 5 1 0 5 5 7
 1 5 5 6 5 3 5 5 5 0 3 5 7 5 0 3 1 5 5 7 0 0 0 0 5 5 2 7 7 5 5 0 3 0 0 0 5
 5 6 0 3]


In [28]:
data_arr = data.values

In [29]:
label_arr

array([0, 0, 1, 2, 0, 0, 3, 0, 1, 4, 3, 0, 5, 0, 0, 0, 0, 4, 1, 6, 1, 7,
       0, 5, 7, 7, 3, 1, 7, 5, 5, 5, 0, 0, 1, 1, 1, 5, 4, 3, 5, 5, 2, 5,
       0, 0, 3, 1, 4, 0, 3, 4, 0, 0, 1, 3, 5, 3, 3, 6, 5, 0, 3, 7, 4, 0,
       5, 5, 7, 5, 2, 7, 0, 5, 6, 5, 3, 5, 4, 0, 5, 2, 5, 2, 4, 2, 3, 0,
       3, 3, 5, 0, 0, 5, 5, 6, 7, 3, 5, 2, 5, 5, 5, 5, 4, 0, 0, 5, 6, 3,
       5, 3, 3, 6, 3, 7, 0, 3, 5, 3, 0, 7, 0, 5, 0, 5, 0, 1, 0, 1, 0, 5,
       7, 0, 7, 3, 5, 0, 0, 1, 5, 0, 2, 1, 5, 4, 1, 7, 0, 5, 0, 0, 5, 2,
       5, 0, 3, 2, 4, 5, 5, 1, 0, 7, 4, 3, 6, 7, 3, 0, 4, 5, 1, 4, 6, 0,
       2, 5, 5, 0, 5, 5, 0, 1, 0, 4, 7, 3, 7, 0, 5, 3, 7, 5, 3, 0, 5, 3,
       1, 5, 1, 0, 1, 0, 0, 3, 0, 7, 3, 7, 6, 1, 5, 1, 0, 5, 5, 3, 7, 0,
       1, 5, 0, 0, 2, 5, 3, 3, 5, 5, 5, 5, 6, 7, 0, 5, 3, 5, 7, 0, 5, 5,
       5, 5, 7, 3, 5, 0, 7, 5, 1, 5, 7, 5, 0, 4, 5, 5, 2, 0, 7, 0, 0, 5,
       0, 5, 0, 5, 3, 5, 5, 0, 6, 3, 3, 4, 2, 5, 0, 7, 3, 4, 7, 0, 4, 5,
       1, 5, 3, 6, 5, 1, 0, 5, 5, 7, 1, 5, 5, 6, 5,

In [30]:
Counter(label_arr)

Counter({np.int64(5): 98,
         np.int64(0): 82,
         np.int64(3): 46,
         np.int64(7): 34,
         np.int64(1): 29,
         np.int64(4): 19,
         np.int64(2): 15,
         np.int64(6): 14})

In [31]:
Counter(y_test)
print(y_test)

                 Label
355          "neurons"
450  "fetal_quiescent"
301           "hybrid"
128  "fetal_quiescent"
12   "fetal_quiescent"
..                 ...
422       "astrocytes"
210          "neurons"
227          "neurons"
341          "neurons"
343      "endothelial"

[113 rows x 1 columns]


In [37]:
X_test

,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,A4GNT,...,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,alignment_not_unique,ambiguous,no_feature,tAKR
GSM1658101_nochipID3.C61.csv,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.258489,0.0,0.781976,0.000000,0.000000,0.000000,8.079006,3.614218,8.163136,0.000000
GSM1658350_nochipID4.C41.csv,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.928907,0.000000,0.016512,0.000000,8.092359,3.539467,8.483096,0.000000
GSM1658047_nochipID2.C50.csv,0.027735,0.00000,0.0,0.0,0.0,0.000000,0.002808,0.000000,0.0,0.0,...,0.002808,0.0,0.002808,0.000000,0.000000,0.000000,7.358178,3.461264,8.661950,0.000000
GSM1658327_nochipID13.C66.csv,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.214791,0.071542,7.802263,3.568970,8.395720,0.000000
GSM1658003_nochipID15.C54.csv,0.000000,0.20202,0.0,0.0,0.0,0.005316,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,1.695764,7.757541,3.113862,8.710790,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1658168_nochipID8.C67.csv,0.003199,0.00000,0.0,0.0,0.0,0.015895,0.000000,0.738460,0.0,0.0,...,0.088861,0.0,0.499134,0.000000,0.031542,0.000000,7.818441,3.392941,8.456992,0.000000
GSM1657942_1772078236.C34.csv,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.245369,0.000000,0.000000,8.335714,3.513105,8.372020,0.000000
GSM1657959_1772078236.C77.csv,0.016409,0.00000,0.0,0.0,0.0,0.454734,0.000000,0.442055,0.0,0.0,...,0.000000,0.0,0.064081,0.013149,0.000000,0.016409,8.072006,2.771102,8.675302,0.000000
GSM1658087_nochipID3.C09.csv,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.011786,0.040657,0.673102,0.299914,8.006585,3.386834,8.453182,0.212781


In [33]:
 Data.TensorDataset(X_test, torch.zeros(len(X_test)))

TypeError: 'int' object is not callable

In [32]:
import torch
dtype = torch.float

# print(X_train)
# X_train=torch.from_numpy(X_train.to_numpy())
# X_test=torch.from_numpy(X_test.to_numpy())
# print(X_test)
# print(y_train)
# y_train = torch.from_numpy(label_arr)
test_data_typical = Data.TensorDataset(X_test, torch.zeros(len(X_test)))
test_data_novel = Data.TensorDataset(torch.from_numpy(X_test_ood.to_numpy()), torch.zeros(len(X_test_ood)))
test_data=torch.utils.data.ConcatDataset([test_data_typical, test_data_novel])
test_loader = Data.DataLoader(dataset=test_data,
                               batch_size=128,
                               num_workers=1)

TypeError: 'int' object is not callable

In [246]:
print(y_train)
print(X_train)
# X_train=torch.from_numpy(X_train.to_numpy())
train_data= Data.TensorDataset(X_train,y_train)

tensor([0, 0, 1, 2, 0, 0, 3, 0, 1, 4, 3, 0, 5, 0, 0, 0, 0, 4, 1, 6, 1, 7, 0, 5,
        7, 7, 3, 1, 7, 5, 5, 5, 0, 0, 1, 1, 1, 5, 4, 3, 5, 5, 2, 5, 0, 0, 3, 1,
        4, 0, 3, 4, 0, 0, 1, 3, 5, 3, 3, 6, 5, 0, 3, 7, 4, 0, 5, 5, 7, 5, 2, 7,
        0, 5, 6, 5, 3, 5, 4, 0, 5, 2, 5, 2, 4, 2, 3, 0, 3, 3, 5, 0, 0, 5, 5, 6,
        7, 3, 5, 2, 5, 5, 5, 5, 4, 0, 0, 5, 6, 3, 5, 3, 3, 6, 3, 7, 0, 3, 5, 3,
        0, 7, 0, 5, 0, 5, 0, 1, 0, 1, 0, 5, 7, 0, 7, 3, 5, 0, 0, 1, 5, 0, 2, 1,
        5, 4, 1, 7, 0, 5, 0, 0, 5, 2, 5, 0, 3, 2, 4, 5, 5, 1, 0, 7, 4, 3, 6, 7,
        3, 0, 4, 5, 1, 4, 6, 0, 2, 5, 5, 0, 5, 5, 0, 1, 0, 4, 7, 3, 7, 0, 5, 3,
        7, 5, 3, 0, 5, 3, 1, 5, 1, 0, 1, 0, 0, 3, 0, 7, 3, 7, 6, 1, 5, 1, 0, 5,
        5, 3, 7, 0, 1, 5, 0, 0, 2, 5, 3, 3, 5, 5, 5, 5, 6, 7, 0, 5, 3, 5, 7, 0,
        5, 5, 5, 5, 7, 3, 5, 0, 7, 5, 1, 5, 7, 5, 0, 4, 5, 5, 2, 0, 7, 0, 0, 5,
        0, 5, 0, 5, 3, 5, 5, 0, 6, 3, 3, 4, 2, 5, 0, 7, 3, 4, 7, 0, 4, 5, 1, 5,
        3, 6, 5, 1, 0, 5, 5, 7, 1, 5, 5,

In [247]:
import torch
import torch.utils.data as Data

def Weighted_Sampling(train_label = None):

    '''

    :param train_label: Input train label (in tensor) to calculate sampling weight
    :return: Pytorch Weighted Sampler
    '''

    class_sample_count = torch.tensor(
        [(train_label == t).sum() for t in torch.unique(train_label, sorted=True)])

    weight = 1. / class_sample_count.float()
    samples_weight = torch.tensor([weight[t] for t in train_label])

    sampler = Data.WeightedRandomSampler(samples_weight, len(samples_weight))
    
    return samples_weight, sampler

In [248]:
print(y_train)
weight, sampler = Weighted_Sampling(y_train)

tensor([0, 0, 1, 2, 0, 0, 3, 0, 1, 4, 3, 0, 5, 0, 0, 0, 0, 4, 1, 6, 1, 7, 0, 5,
        7, 7, 3, 1, 7, 5, 5, 5, 0, 0, 1, 1, 1, 5, 4, 3, 5, 5, 2, 5, 0, 0, 3, 1,
        4, 0, 3, 4, 0, 0, 1, 3, 5, 3, 3, 6, 5, 0, 3, 7, 4, 0, 5, 5, 7, 5, 2, 7,
        0, 5, 6, 5, 3, 5, 4, 0, 5, 2, 5, 2, 4, 2, 3, 0, 3, 3, 5, 0, 0, 5, 5, 6,
        7, 3, 5, 2, 5, 5, 5, 5, 4, 0, 0, 5, 6, 3, 5, 3, 3, 6, 3, 7, 0, 3, 5, 3,
        0, 7, 0, 5, 0, 5, 0, 1, 0, 1, 0, 5, 7, 0, 7, 3, 5, 0, 0, 1, 5, 0, 2, 1,
        5, 4, 1, 7, 0, 5, 0, 0, 5, 2, 5, 0, 3, 2, 4, 5, 5, 1, 0, 7, 4, 3, 6, 7,
        3, 0, 4, 5, 1, 4, 6, 0, 2, 5, 5, 0, 5, 5, 0, 1, 0, 4, 7, 3, 7, 0, 5, 3,
        7, 5, 3, 0, 5, 3, 1, 5, 1, 0, 1, 0, 0, 3, 0, 7, 3, 7, 6, 1, 5, 1, 0, 5,
        5, 3, 7, 0, 1, 5, 0, 0, 2, 5, 3, 3, 5, 5, 5, 5, 6, 7, 0, 5, 3, 5, 7, 0,
        5, 5, 5, 5, 7, 3, 5, 0, 7, 5, 1, 5, 7, 5, 0, 4, 5, 5, 2, 0, 7, 0, 0, 5,
        0, 5, 0, 5, 3, 5, 5, 0, 6, 3, 3, 4, 2, 5, 0, 7, 3, 4, 7, 0, 4, 5, 1, 5,
        3, 6, 5, 1, 0, 5, 5, 7, 1, 5, 5,

In [249]:
weight

tensor([0.0122, 0.0122, 0.0345, 0.0667, 0.0122, 0.0122, 0.0217, 0.0122, 0.0345,
        0.0526, 0.0217, 0.0122, 0.0102, 0.0122, 0.0122, 0.0122, 0.0122, 0.0526,
        0.0345, 0.0714, 0.0345, 0.0294, 0.0122, 0.0102, 0.0294, 0.0294, 0.0217,
        0.0345, 0.0294, 0.0102, 0.0102, 0.0102, 0.0122, 0.0122, 0.0345, 0.0345,
        0.0345, 0.0102, 0.0526, 0.0217, 0.0102, 0.0102, 0.0667, 0.0102, 0.0122,
        0.0122, 0.0217, 0.0345, 0.0526, 0.0122, 0.0217, 0.0526, 0.0122, 0.0122,
        0.0345, 0.0217, 0.0102, 0.0217, 0.0217, 0.0714, 0.0102, 0.0122, 0.0217,
        0.0294, 0.0526, 0.0122, 0.0102, 0.0102, 0.0294, 0.0102, 0.0667, 0.0294,
        0.0122, 0.0102, 0.0714, 0.0102, 0.0217, 0.0102, 0.0526, 0.0122, 0.0102,
        0.0667, 0.0102, 0.0667, 0.0526, 0.0667, 0.0217, 0.0122, 0.0217, 0.0217,
        0.0102, 0.0122, 0.0122, 0.0102, 0.0102, 0.0714, 0.0294, 0.0217, 0.0102,
        0.0667, 0.0102, 0.0102, 0.0102, 0.0102, 0.0526, 0.0122, 0.0122, 0.0102,
        0.0714, 0.0217, 0.0102, 0.0217, 

In [250]:
train_loader= Data.DataLoader(dataset=train_data,batch_size=32,sampler = sampler,num_workers=1)

In [251]:
from scNovel import classifier

In [252]:
input_size = X_train.shape[1]
num_class = len(status_dict)
iteration=3000
count_iteration=0

model = classifier(input_size, num_class)

In [253]:
print(model)

classifier(
  (layer1): Linear(in_features=22088, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (elu1): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.5, inplace=False)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (elu2): ELU(alpha=1.0)
  (dropout2): Dropout(p=0.5, inplace=False)
  (layer3): Linear(in_features=64, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (elu3): ELU(alpha=1.0)
  (dropout3): Dropout(p=0.1, inplace=False)
  (layer4): Linear(in_features=32, out_features=8, bias=True)
)


In [254]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
total_step = len(train_loader)

In [255]:
import time
print("--------Start annotating----------")
start = time.perf_counter()

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# if processing_unit == 'cpu':
#     device = torch.device('cpu')
# elif processing_unit == 'gpu':
#     if torch.cuda.is_available():
#         device = torch.device('cuda')
#     else:
#         device = torch.device('cpu')
#         print("No GPUs are available on your server.")

print("Computational unit be used is:", device)

model.to(device)

model.train()
while count_iteration<iteration:
    for step, (batch_x, batch_y) in enumerate(train_loader):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        train_loss = criterion(outputs, batch_y)

        # Backward and optimize
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        count_iteration+=1
        if count_iteration>=iteration:
            break
    print("Finish {}/{}".format(float(count_iteration),iteration))
    # a = "=" * (count_iteration + 1)
    # b = "." * (iteration - count_iteration - 1)
    # c = ((count_iteration + 1) / iteration) * 100
    # dur = time.perf_counter() - start
    # print("\r{:^3.0f}%[{}->{}]{:.2f}s".format(c, a, b, dur), end='')
    # time.sleep(0.1)


print("\n--------Annotation Finished----------")

--------Start annotating----------
Computational unit be used is: cuda
Finish 11.0/3000
Finish 22.0/3000
Finish 33.0/3000
Finish 44.0/3000
Finish 55.0/3000
Finish 66.0/3000
Finish 77.0/3000
Finish 88.0/3000
Finish 99.0/3000
Finish 110.0/3000
Finish 121.0/3000
Finish 132.0/3000
Finish 143.0/3000
Finish 154.0/3000
Finish 165.0/3000
Finish 176.0/3000
Finish 187.0/3000
Finish 198.0/3000
Finish 209.0/3000
Finish 220.0/3000
Finish 231.0/3000
Finish 242.0/3000
Finish 253.0/3000
Finish 264.0/3000
Finish 275.0/3000
Finish 286.0/3000
Finish 297.0/3000
Finish 308.0/3000
Finish 319.0/3000
Finish 330.0/3000
Finish 341.0/3000
Finish 352.0/3000
Finish 363.0/3000
Finish 374.0/3000
Finish 385.0/3000
Finish 396.0/3000
Finish 407.0/3000
Finish 418.0/3000
Finish 429.0/3000
Finish 440.0/3000
Finish 451.0/3000
Finish 462.0/3000
Finish 473.0/3000
Finish 484.0/3000
Finish 495.0/3000
Finish 506.0/3000
Finish 517.0/3000
Finish 528.0/3000
Finish 539.0/3000
Finish 550.0/3000
Finish 561.0/3000
Finish 572.0/3000
Fi

In [256]:
from scNovel.score_functions import get_ood_scores
model.eval()
score_function="sim"

test_score = get_ood_scores(test_loader, model, score_function, device=device)

auroc=sklearn.metrics.roc_auc_score(true_label,test_score)
print(auroc)



0.9175884955752213
